<a href="https://colab.research.google.com/github/jolex007/COS_DL/blob/master/1_HomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Задание по машинке

In [1]:
import numpy as np
import os
import cv2

## Инициализация модели

In [2]:
def init_model(input_size=256):
    np.random.seed(2)
    w = np.random.randn(input_size ** 2, 1) * 0.05
    b = 0
    return w, b

In [3]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

## Forward & Backward propogation

In [4]:
def propagate(w, b, X, Y):
    """
    Подсчет текущего предсказания (оно же forward propagation) и градиента функции ошибки (оно же backward propagation)

    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)

    Return:
    cost -- текущая функция ошибки
    dw -- градиент функции ошибки по w
    db -- градиент функции ошибки по b (по сути производная по b)
    
    """
    
    m = X.shape[1]
    

    A = sigmoid(w.T @ X + b)

    cost = (-1. / m) * (Y * np.log(A) + (1 - Y) * np.log(1 - A)).sum()
    
    dw = (1. / m) * X @ (A - Y).T
    db = (1. / m) * (A - Y).sum()

    grads = {"dw": dw,
             "db": db}

    return grads, cost

## Шаг обновления весов

In [5]:
def optimize(w, b, X, Y, num_iterations, learning_rate=0.002, print_cost=False):
    """
    Оптимизация с помощью простого градиентного спуска
    
    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)
    num_iterations -- кол-во итераций алгоритма оптимизации
    learning_rate -- коэффициент learning rate
    print_cost -- True, если хотите выводить функцию ошибки на каждых 100 итерациях
    
    Returns:
    params -- словарь, содержащий w и b
    grads -- словарь, содержащий градиенты функции ошибки по w и b соответственно
    costs -- массив (list) со значением функции ошибки для каждой итерации (так делают для визуализации)
    
    Подсказка:
    
        1) Используйте ранее написанную функцию propagate().
        2) Обновляйте параметры w и b согласно формуле 10.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        ###Напишите значения для градиентов и функции ошибки
        grads, cost = propagate(w, b, X, Y)
        
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # обновление параметров
        
        ### START CODE HERE ###
        w = w - learning_rate * dw
        b = b - learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

## Предсказание модели

In [6]:
def predict(w, b, X):
    '''
    Inputs:
    w
    b
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    
    Returns:
    Y_prediction
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(w.T @ X + b)
    ### END CODE HERE ###
    
    for i in range(A.shape[1]):
        # Установите порог, выше которого считаем, что модель выдает 1, а ниже - ноль
        ### START CODE HERE ###
        if A[0][i] <= 0.7:
            Y_prediction[0][i] = 0
        else:
            Y_prediction[0][i] = 1
        ### END CODE HERE ###
    
    
    return Y_prediction

In [7]:
def model(X, Y, num_iterations=10000, print_cost=False):
    w, b = init_model()
    params, _, _ = optimize(w, b, X, Y, num_iterations, print_cost=print_cost)
    
    return params["w"], params["b"]

## Обработка датасета

In [8]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
def read_files(X, Y, path, ans):
    files = os.listdir(path)
    for name in files:
        img = cv2.imread(path + '/' + name, 0)
        if img.shape != 0:
            img = cv2.resize(img, (256, 256))
            vect = img.reshape(1, 256 ** 2)
            vect = vect / 255.
            X = vect if (X is None) else np.vstack((X, vect)) 
            Y = np.append(Y, ans)
    return X, Y

path = "gdrive/My Drive/COS/lesson1_dataset"
# os.listdir("content/gdrive/")
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

X = X.T

## Запуск модели

In [10]:
w, b = model(X, Y, num_iterations=1000, print_cost=True)

Cost after iteration 0: 0.801620
Cost after iteration 100: 0.128781
Cost after iteration 200: 0.077143
Cost after iteration 300: 0.056444
Cost after iteration 400: 0.045118
Cost after iteration 500: 0.037910
Cost after iteration 600: 0.032890
Cost after iteration 700: 0.029177
Cost after iteration 800: 0.026310
Cost after iteration 900: 0.024023


In [11]:
result = predict(w, b, X)

In [12]:
print("My Results: ", result)
print("Real Results: ", Y)

T = 0
All = 0

for i in range(result.shape[1]):
    if result[0][i] == Y[i]:
        T += 1
    All += 1

print("Total accuracy: %f%%" %(T / All * 100))

My Results:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Real Results:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Total accuracy: 100.000000%
